In [83]:
import pandas as pd

# ===========================================
# 1. 데이터 불러오기 & 전처리
# ===========================================
df_temp = pd.read_csv("일별_기온_강수량_일조15~24.csv")
df_bloom = pd.read_csv("북숭아_개화_만발_보정.csv")

df_temp["일시"] = pd.to_datetime(df_temp["일시"]).dt.date
df_bloom["개화"] = pd.to_datetime(df_bloom["개화"], errors="coerce").dt.date
df_bloom["만발"] = pd.to_datetime(df_bloom["만발"], errors="coerce").dt.date
df_bloom["복숭아_발아"] = pd.to_datetime(df_bloom["복숭아_발아"], errors="coerce").dt.date

# ===========================================
# 2. 시도명 매핑
# ===========================================
region_map = {
    "서울":"서울특별시","인천":"인천광역시","수원":"경기도","북춘천":"강원도","북강릉":"강원도",
    "대전":"대전광역시","대구":"대구광역시","부산":"부산광역시","울산":"울산광역시",
    "광주":"광주광역시","포항":"경상북도","안동":"경상북도","전주":"전라북도",
    "창원":"경상남도","여수":"전라남도","목포":"전라남도","흑산도":"전라남도",
    "백령도":"인천광역시","울릉도":"경상북도",
    "서귀포":"제주도","제주":"제주도","홍성":"충청남도","청주":"충청북도",
    "순천":"전라남도"
}
df_bloom["시도명"] = df_bloom["지점"].map(region_map)

# ===========================================
# 3. 대전 기상 데이터 미리 분리
# ===========================================
df_temp_daejeon = df_temp[df_temp["지점명"] == "대전"]

records = []

# ===========================================
# 4. 기존 df_bloom 데이터 기반 냉해 스트레스 계산
# ===========================================
for idx, row in df_bloom.iterrows():
    region = row["지점"]
    year = int(row["연도"])
    sido = row["시도명"]

    start_date = row["복숭아_발아"]
    end_date = row["만발"]

    if pd.isna(start_date) or pd.isna(end_date):
        continue

    # 충남 2015~2017 → 대전 사용
    if sido == "충청남도" and year in [2015, 2016, 2017]:
        subset = df_temp_daejeon[
            (df_temp_daejeon["일시"] >= start_date) &
            (df_temp_daejeon["일시"] <= end_date)
        ]

    # 세종 → 전체연도 대전 사용
    elif sido == "세종특별자치시":
        subset = df_temp_daejeon[
            (df_temp_daejeon["일시"] >= start_date) &
            (df_temp_daejeon["일시"] <= end_date)
        ]

    # 기본 지역 처리
    else:
        subset = df_temp[
            (df_temp["지점명"] == region) &
            (df_temp["일시"] >= start_date) &
            (df_temp["일시"] <= end_date)
        ]

    cold_days = (subset["최저기온(°C)"] <= 0).sum()

    records.append({
        "시도명": sido,
        "연도": year,
        "총합_냉해_스트레스_일수": int(cold_days)
    })


# ===========================================
# 5. 충청남도 2015~2017 강제 추가 (대전 날짜+대전 기상 사용)
# ===========================================
daejeon_rows = df_bloom[df_bloom["시도명"] == "대전광역시"]

for y in [2015, 2016, 2017]:

    daejeon_year = daejeon_rows[daejeon_rows["연도"] == y]
    if len(daejeon_year) == 0:
        continue

    start_date = daejeon_year.iloc[0]["복숭아_발아"]
    end_date   = daejeon_year.iloc[0]["만발"]

    subset = df_temp_daejeon[
        (df_temp_daejeon["일시"] >= start_date) &
        (df_temp_daejeon["일시"] <= end_date)
    ]

    cold_days = (subset["최저기온(°C)"] <= 0).sum()

    records.append({
        "시도명": "충청남도",
        "연도": y,
        "총합_냉해_스트레스_일수": int(cold_days)
    })


# ===========================================
# 6. 세종특별자치시 전체 연도 강제 추가
# ===========================================
years = sorted(df_bloom["연도"].unique())

for y in years:
    # 같은 연도의 다른 지역에서 날짜 하나 선택 (대전 기준 아님)
    sample_rows = df_bloom[df_bloom["연도"] == y]
    if len(sample_rows) == 0:
        continue

    start_date = sample_rows.iloc[0]["복숭아_발아"]
    end_date   = sample_rows.iloc[0]["만발"]

    subset = df_temp_daejeon[
        (df_temp_daejeon["일시"] >= start_date) &
        (df_temp_daejeon["일시"] <= end_date)
    ]

    cold_days = (subset["최저기온(°C)"] <= 0).sum()

    records.append({
        "시도명": "세종특별자치시",
        "연도": int(y),
        "총합_냉해_스트레스_일수": int(cold_days)
    })


# ===========================================
# 7. 중복 제거 및 정렬
# ===========================================
df_result = pd.DataFrame(records)
df_result = df_result.drop_duplicates(subset=["시도명", "연도"], keep="first")
df_result = df_result.sort_values(by=["시도명", "연도"])

df_result.to_csv("냉해_스트레스.csv", index=False)

print(df_result)


      시도명    연도  총합_냉해_스트레스_일수
0     강원도  2015              0
1     강원도  2016              0
2     강원도  2017              0
3     강원도  2018              0
4     강원도  2019              0
..    ...   ...            ...
217  충청북도  2020              2
218  충청북도  2021              0
219  충청북도  2022              0
220  충청북도  2023              1
221  충청북도  2024              3

[170 rows x 3 columns]
